In [ ]:
!pip install -q datasets
!pip install -q pandas
!pip install -q transformers
!pip install -q tensorflow
!pip install -q huggingface_hub

^C
ERROR: Operation cancelled by user


In [1]:

import pandas as pd
import numpy as np

import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset
from huggingface_hub import notebook_login

2023-09-22 10:46:08.262389: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7630] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-22 10:46:08.262439: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-22 10:46:08.262503: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-22 10:46:08.446011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 10:46:13.943208: W tensorflow/compiler/

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
notebook_login()

In [4]:
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
model = TFAutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased",)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
instrucciones = load_dataset('Herreera1/jovenes') ## se puede hacer mejor xd

In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)
datos_encoded = instrucciones.map(tokenize, batched=True, batch_size=None) ## gracias dios de la india

In [8]:
datos_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
BATCH_SIZE = 4

def order(inp):

    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]
train_dataset = tf.data.Dataset.from_tensor_slices(datos_encoded['train'][:])
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(datos_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [11]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [12]:
classifier = BERTForClassification(model, num_classes=9)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)


In [13]:
history = classifier.fit(
    train_dataset,
    epochs=20
)

Epoch 1/20


2023-09-22 10:49:02.616961: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5e792890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 10:49:02.616981: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1050, Compute Capability 6.1
2023-09-22 10:49:02.621935: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 10:49:02.767668: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:427] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-09-22 10:49:02.769074

FailedPreconditionError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_199 defined at (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,

  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,

  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)

  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,

  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)

  File "/home/laura/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()

  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,

  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_g

In [ ]:
classifier.evaluate(test_dataset)

16/16 [==============================] - 11s 432ms/step - loss: 2.0530 - accuracy: 0.5484


[2.0530195236206055, 0.5483871102333069]

In [ ]:
import numpy as np

# Supongamos que tienes un texto para clasificar
texto = "que hay en esa direccion y seña derecha"

# Tokenizar el texto
inputs = tokenizer(texto, padding=True, truncation=True, return_tensors="tf")

# Obtener la predicción usando el modelo
predicciones = classifier(inputs)

# Obtener las clases predichas
clases_predichas = np.argmax(predicciones, axis=1)

# Imprimir las clases predichas
print("Clase predicha:", clases_predichas)

Clase predicha: [7]
